In [1]:
import awkward as ak
import hist
import numpy as np
from coffea import util as cutil
from coffea.lookup_tools.dense_lookup import dense_lookup
from coffea.nanoevents import NanoAODSchema, NanoEventsFactory

btagWPs = {
    "deepJet": {
        "2016preVFP_UL": {
            "L": 0.0508,
            "M": 0.2598,
            "T": 0.6502,
        },
        "2016postVFP_UL": {
            "L": 0.0480,
            "M": 0.2489,
            "T": 0.6377,
        },
        "2017_UL": {
            "L": 0.0532,
            "M": 0.3040,
            "T": 0.7476,
        },
        "2018_UL": {
            "L": 0.0490,
            "M": 0.2783,
            "T": 0.7100,
        },
    },
    "deepCSV": {
        "2016preVFP_UL": {
            "L": 0.2027,
            "M": 0.6001,
            "T": 0.8819,
        },
        "2016postVFP_UL": {
            "L": 0.1918,
            "M": 0.5847,
            "T": 0.8767,
        },
        "2017_UL": {
            "L": 0.1355,
            "M": 0.4506,
            "T": 0.7738,
        },
        "2018_UL": {
            "L": 0.1208,
            "M": 0.4168,
            "T": 0.7665,
        },
    },
}

# single TT files to derive efficiency

taggerBranch = {"deepJet": "btagDeepFlavB", "deepCSV": "btagDeepB"}


file1 = "/uscms_data/d3/jieun201/YOURWORKINGAREA/nano_mc2017_227.root"

events = NanoEventsFactory.from_root(
    file1,
entry_stop=100_000, schemaclass=NanoAODSchema,
).events()
year = '2017_UL'

#events = file1.events()

# b-tagging only applied for jets with |eta| < 2.5
phasespace_cuts = (events.Jet.pt > 30) & (abs(events.Jet.eta) < 2.5)
jets = ak.flatten(events.Jet[phasespace_cuts])

bins = [40,60,80,100,120,140,160,180,200,220,240,260,280,300,1000]


#for tag in ["deepJet", "deepCSV"]:
for tag in ["deepJet"]:
    #for wp in ["L", "M", "T"]:
    for wp in ["M"]:
        efficiencyinfo = (
            hist.Hist.new.Variable(bins, name="pt")
            .Reg(4, 0, 2.5, name="abseta")
            .IntCat([0, 4, 5], name="flavor")
            .Bool(name="passWP")
            .Double()
            .fill(
                pt=jets.pt,
                abseta=abs(jets.eta),
                flavor=jets.hadronFlavour,
                passWP=jets[taggerBranch[tag]] > btagWPs[tag][year][wp],
            )
        )
        eff = efficiencyinfo[{"passWP": True}] / efficiencyinfo[{"passWP": sum}]
        efflookup = dense_lookup(eff.values(), [ax.edges for ax in eff.axes])
        #print(tag, wp, efflookup(np.array([42, 60]), 0.2, 2))

    #    cutil.save(efflookup, f"btageff_{tag}_{wp}_{year}.coffea")

/home/jieun201/.local/lib/python3.10/site-packages/coffea/nanoevents/mapping/uproot.py:86: UserWarning: Found duplicate branch FatJetAK15SubJet_nBHadrons in <TTree 'Events' (1975 branches) at 0x7f14ff3220e0>, taking first instance
  warnings.warn(
/home/jieun201/.local/lib/python3.10/site-packages/coffea/nanoevents/mapping/uproot.py:86: UserWarning: Found duplicate branch FatJetAK15SubJet_nCHadrons in <TTree 'Events' (1975 branches) at 0x7f14ff3220e0>, taking first instance
  warnings.warn(
/home/jieun201/.local/lib/python3.10/site-packages/coffea/nanoevents/mapping/uproot.py:86: UserWarning: Found duplicate branch FatJetAK15_nBHadrons in <TTree 'Events' (1975 branches) at 0x7f14ff3220e0>, taking first instance
  warnings.warn(
/home/jieun201/.local/lib/python3.10/site-packages/coffea/nanoevents/mapping/uproot.py:86: UserWarning: Found duplicate branch FatJetAK15_nCHadrons in <TTree 'Events' (1975 branches) at 0x7f14ff3220e0>, taking first instance
  warnings.warn(
/home/jieun201/.loca

In [2]:
print(eff)

Hist(
  Variable([40, 60, 80, 100, 120, 140, 160, 180, 200, 220, 240, 260, 280, 300, 1000], name='pt'),
  Regular(4, 0, 2.5, name='abseta'),
  IntCategory([0, 4, 5], name='flavor'),
  storage=Double()) # Sum: 55.46420591799028 (nan with flow)


In [3]:
#s_2d = eff.project("pt","abseta")
#s_2d[{"flavor": 4j}]

In [4]:
eff_4=eff[{"flavor": 4j}]

In [5]:
eff_4.values()

array([[0.15542522, 0.16011561, 0.16324921, 0.16778976],
       [0.12838307, 0.14355828, 0.14209275, 0.13510747],
       [0.14369345, 0.12891114, 0.14005602, 0.12678571],
       [0.12991453, 0.13702624, 0.16455696, 0.10909091],
       [0.13517241, 0.15614035, 0.15549598, 0.13089005],
       [0.1292517 , 0.11728395, 0.12068966, 0.16393443],
       [0.16491228, 0.10096154, 0.10596026, 0.08      ],
       [0.15555556, 0.15625   , 0.13684211, 0.18367347],
       [0.144     , 0.14678899, 0.16666667, 0.15789474],
       [0.13      , 0.1641791 , 0.06060606, 0.33333333],
       [0.1875    , 0.225     , 0.11538462, 0.13333333],
       [0.19444444, 0.03846154, 0.23529412, 0.2       ],
       [0.25      , 0.10714286, 0.41666667, 0.        ],
       [0.21538462, 0.18811881, 0.21276596, 0.25      ]])

In [6]:
eff_5=eff[{"flavor": 5j}]

In [7]:
eff_5.values()

array([[0.83050353, 0.81207942, 0.77918053, 0.7590561 ],
       [0.85246882, 0.83492289, 0.80030143, 0.76858345],
       [0.8616207 , 0.83908379, 0.80067265, 0.76224662],
       [0.84657483, 0.84868421, 0.8094195 , 0.77671334],
       [0.86314152, 0.84647147, 0.82132222, 0.78803641],
       [0.85327992, 0.85796178, 0.81313131, 0.79107981],
       [0.87177419, 0.86258776, 0.80895009, 0.81069959],
       [0.87231353, 0.8573913 , 0.80886427, 0.79699248],
       [0.84989858, 0.82471264, 0.82513661, 0.72619048],
       [0.87579618, 0.87878788, 0.76612903, 0.85185185],
       [0.9       , 0.8452381 , 0.81372549, 0.76190476],
       [0.83229814, 0.85714286, 0.84375   , 0.8       ],
       [0.87826087, 0.85483871, 0.78787879, 0.75      ],
       [0.82712766, 0.8057554 , 0.77310924, 0.72      ]])

In [8]:
eff_0=eff[{"flavor": 0j}]

In [9]:
eff_0.values()

array([[0.01253946, 0.01483606, 0.01753349, 0.02411893],
       [0.00888536, 0.00821205, 0.01207221, 0.01240741],
       [0.00715878, 0.00920583, 0.0094038 , 0.01627604],
       [0.00583544, 0.00922935, 0.01330377, 0.01469788],
       [0.01077727, 0.00631501, 0.0133833 , 0.01061947],
       [0.00862534, 0.00769686, 0.01481481, 0.01651652],
       [0.00415628, 0.00756859, 0.01530612, 0.01882353],
       [0.00617284, 0.01420455, 0.00769231, 0.01020408],
       [0.0070922 , 0.01077586, 0.01634877, 0.02252252],
       [0.01373626, 0.00636943, 0.01185771, 0.00680272],
       [0.00343643, 0.01276596, 0.01829268, 0.03296703],
       [0.01129944, 0.00657895, 0.        , 0.01639344],
       [0.00606061, 0.00763359, 0.01      , 0.03448276],
       [0.02291326, 0.02056075, 0.04347826, 0.05586592]])